In [ ]:
# Mount Google Drive Folder (/content/drive/My Drive/Colab)
from google.colab import drive
drive.mount('/content/drive')

# import user defined files 
import sys
import_dir ="/content/drive/My Drive/Colab Notebooks/3. Semantic Segmentation"
sys.path.insert(0, import_dir)

# import ipynb files
!pip install import_ipynb 
import import_ipynb

# Import ipynb in different directory
#%run '/content/drive/My Drive/Colab Notebooks/3. Semantic Segmentation/dataset.ipynb'

# import easydict instead of using argparse (argparse is not usuable for ipynb)
import easydict 
global _args
_args = easydict.EasyDict({"config": '/content/drive/My Drive/Colab Notebooks/3. Semantic Segmentation/config/isic.yaml'})

# Debugging Tool
import pdb

Mounted at /content/drive
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=8e53bee728bb8f3982dcfc49f4a5729d11c529bc5e7bbd53b7eaaf84b6666a8f
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [ ]:
# Download & Unzip Dataset (Move from google drive to local)
%cp -r '/content/drive/My Drive/Colab Notebooks/Dataset/ISIC/ISIC.zip' '/content/sample_data'
!unzip '/content/sample_data/ISIC.zip' 


Archive:  /content/sample_data/ISIC.zip
   creating: ISIC_Testset/
   creating: ISIC_Testset/image/
  inflating: ISIC_Testset/image/ISIC_0000264.jpg  
  inflating: ISIC_Testset/image/ISIC_0000279.jpg  
  inflating: ISIC_Testset/image/ISIC_0000293.jpg  
  inflating: ISIC_Testset/image/ISIC_0010017.jpg  
  inflating: ISIC_Testset/image/ISIC_0010078.jpg  
  inflating: ISIC_Testset/image/ISIC_0010329.jpg  
  inflating: ISIC_Testset/image/ISIC_0010351.jpg  
  inflating: ISIC_Testset/image/ISIC_0010364.jpg  
  inflating: ISIC_Testset/image/ISIC_0010473.jpg  
  inflating: ISIC_Testset/image/ISIC_0010571.jpg  
   creating: ISIC_Testset/label/
  inflating: ISIC_Testset/label/ISIC_0000264_segmentation.png  
  inflating: ISIC_Testset/label/ISIC_0000279_segmentation.png  
  inflating: ISIC_Testset/label/ISIC_0000293_segmentation.png  
  inflating: ISIC_Testset/label/ISIC_0010017_segmentation.png  
  inflating: ISIC_Testset/label/ISIC_0010078_segmentation.png  
  inflating: ISIC_Testset/label/ISIC_

In [ ]:
# Initilize Additional parameters
train_root = "/content/ISIC_Trainset"
test_root = "/content/ISIC_Testset"

In [ ]:
import os
import pdb
import argparse
import numpy as np
import torch
import torch.nn as nn

import models
from utils import dataset, config, utils, dataset_utils

In [ ]:
def get_parser(args): # args는 config파일 위치
    '''
    # Removed Code (Because ipynb file does not support argparse)
    # Even removed, these lines are substitued by easydict code

    parser = argparse.ArgumentParser(description='PyTorch Semantic Segmentation')
    parser.add_argument('--config', type=str, default='config/cityscapes.yaml')
    args = parser.parse_args()
    '''
    
    assert args.config is not None # 예외처리 얘가 False가 되면 에러난다.

    '''
    yaml파일 읽어서 key, value로 읽어들인 값을 {}에 집어넣고 그대로 딕셔너리 리턴하지 않고 
    하나를 더해준다.
    ConfigNode 클래스를 하나 더 만들고 딕셔너리 형태를 다시 argparse형태로 바꿔주는 부분
    '''
    cfg = config.load_cfg_from_cfg_file(args.config) # utils<config.py<load_cfg_from_cfg_file
    return cfg

In [ ]:
def check(args): #리스트 안에 있으면 True 없으면 False가 나고 에러로 떨어진다.
    assert args.data_name in ['crack', 'cityscapes', 'isic']
    assert args.classes >= 1
    assert args.model in ['unet32', 'unet64', 'unet128',
                          'deeplab34','deeplab50','deeplab101',
                          'pspnet18', 'pspnet34', 'pspnet50']
    assert args.loss in ['bce','ce','dice']

In [ ]:
def main():
    args = get_parser(_args) # isic.yaml에 있는 내용을 dict로 반환
    check(args) # args가 제대로 된 dict인지 확인
    os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(str(x) for x in args.gpu) # gpu 세팅방법.
    main_worker(args)

In [ ]:
def main_worker(argss):
    global args
    args = argss

    ##### Model #####

    model = models.load_model(args) 
    pdb.set_trace() # 막아야 됨
    optimizer = torch.optim.Adam(model.parameters(), lr=args.base_lr)
    
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    best_loss = 999.0

    if len(args.gpu) > 1:
        model = torch.nn.DataParallel(model.cuda())
    else:
        model = model.cuda()
    print(f"=> creating {args.model} (classes:{args.classes})")

    os.makedirs(args.ckpt_root, exist_ok=True) # ckpt_root 없으면(최초 실행시) 생성

    
    if args.weight: # resume기능
        if os.path.isfile(args.weight):
            ckpt = torch.load(args.weight, map_location=lambda storage, loc: storage.cuda())
            args.start_epoch = ckpt['epoch']
            model.load_state_dict(ckpt['state_dict'], strict=False)
            optimizer.load_state_dict(ckpt['optimizer'])
            print(f"=> loaded checkpoint '{args.weight}' (epoch {ckpt['epoch']})") 
        else:
            print(f"=> no weight found at '{args.weight}'")


    ##### Data #####
    criterion, transforms, _, _, _ = dataset_utils.dataset_utils(args) # 후처리 관련

    train_loader = torch.utils.data.DataLoader(
                    dataset.Dataset(mode='train', data_root=args.train_root, data_list=args.train_list,
                                    transform=transforms[0], label_transform=transforms[3]),
                    batch_size=args.batch_size, shuffle=True, num_workers=args.workers,
                    pin_memory=True, drop_last=True)

    val_loader = torch.utils.data.DataLoader(
                    dataset.Dataset(mode='val', data_root=args.val_root, data_list=args.val_list,
                                    transform=transforms[1], label_transform=transforms[3]),
                    batch_size=args.batch_size, shuffle=False, num_workers=args.workers,
                    pin_memory=True)


    ##### Train & Validation #####
    del_ckpt = None
    for epoch in range(args.start_epoch, args.epochs):
        loss_train = train(train_loader, model, criterion,
                                        optimizer, scheduler, epoch)
        loss_val = validate(val_loader, model, criterion, epoch)

        if best_loss > loss_val:
            best_loss = loss_val
            if del_ckpt:
                os.remove(del_ckpt)
            filename = f'{args.ckpt_root}/{args.data_name}_{args.model}_{epoch+1}.pth'
            torch.save({'epoch': epoch+1,
                        'state_dict': model.state_dict(),
                        'optimizer': optimizer.state_dict()}, filename)
            del_ckpt = filename
            print(f'Saving checkpoint to: {filename}')

In [ ]:
def train(train_loader, model, criterion, optimizer, scheduler, epoch): # 이부분은 이미 알고 있는 코드이다.
    print(f'\nMode: Train | Epoch: {epoch+1}/{args.epochs} | Model: {args.model} | Data: {args.data_name}')
    loss_meter = utils.AverageMeter()

    torch.set_grad_enabled(True)
    model.train()
    for i, data in enumerate(train_loader):
        img, label = data[0].cuda(), data[1].cuda()
        output = model(img)
        loss = criterion(output, label.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_meter.update(loss.item(), img.size(0))
        # utils.progress_bar(i, len(train_loader), f'Loss:{loss_meter.avg:.4f}')
    print(f'Loss:{loss_meter.avg:.4f}')
    scheduler.step()
    torch.cuda.empty_cache()
    return loss_meter.avg

In [ ]:
def validate(val_loader, model, criterion, epoch):
    print(f'\nMode: Validation  |  Epoch: {epoch+1}/{args.epochs}  |  Model: {args.model}')
    loss_meter = utils.AverageMeter()

    torch.set_grad_enabled(False)
    model.eval()
    for i, data in enumerate(val_loader):
        img, label = data[0].cuda(), data[1].cuda()
        output = model(img)
        loss = criterion(output, label.long())
        loss_meter.update(loss.item(), img.size(0))
        # utils.progress_bar(i, len(val_loader), f'Loss:{loss_meter.avg:.4f}')
    print(f'Loss:{loss_meter.avg:.4f}')
    torch.cuda.empty_cache()
    return loss_meter.avg

In [ ]:
if __name__ == '__main__':
    main()


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 332, in set_trace
    sys.settrace(self.trace_dispatch)



> <ipython-input-9-aeee9344e28c>(9)main_worker()
-> optimizer = torch.optim.Adam(model.parameters(), lr=args.base_lr)
UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (conv): Sequential(
      (0): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(64